<a href="https://colab.research.google.com/github/avk4714/DL_Projects/blob/master/ok_deep_ensembes_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [0]:
learning_rate = 0.01
input_dim = 6
output_dim = 2*4
hidden_units = 500
networks_num = 5
num_iter = 2000 # epochs

In [0]:
class Net(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_units):
    super(Net,self).__init__()
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.hidden_units = hidden_units
    self.model = torch.nn.Sequential(
        torch.nn.Linear(self.input_dim, self.hidden_units, bias=True),
        torch.nn.ReLU(),
        torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
        torch.nn.ReLU(),
        torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
        torch.nn.ReLU(),
        torch.nn.Linear(self.hidden_units, self.hidden_units, bias=True),
        torch.nn.ReLU(),
        torch.nn.Linear(self.hidden_units, self.output_dim, bias=True),
    )
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
  
  def forward(self, inputs):
    out = self.model(inputs)
    mean, var = torch.split(out, self.output_dim//2, dim=1)
    var = F.softplus(var) + 1e-7 # add a minimum variance for numerical stability
    return mean, var

In [0]:
def NLL(mean, var, truth):
   """ Compute the negative log likelihood """
   diff = torch.sub(truth, mean)
   loss = torch.mean(torch.div(diff**2, 2*var) + 0.5*torch.log(var))
   return loss.sum()

# True data

In [0]:
from numpy import genfromtxt

x = genfromtxt('train_x_data_2.csv', delimiter=',')
y = genfromtxt('train_y_data_2.csv', delimiter=',')
train_x = x [:1000]
train_y = y [:1000]

# Training the ensemble

In [0]:
model0 = Net(input_dim, output_dim, hidden_units)
model1 = Net(input_dim, output_dim, hidden_units)
model2 = Net(input_dim, output_dim, hidden_units)
model3 = Net(input_dim, output_dim, hidden_units)
model4 = Net(input_dim, output_dim, hidden_units)
ensemble = [model0, model1, model2, model3, model4]

for t in range(num_iter):
    mean_s = []
    var_s = []
    losses = []
    for i in range(networks_num):
        model = ensemble[i]
        mean, var = model(torch.tensor(train_x).float())
        loss = NLL(mean, var, torch.tensor(train_y).float())
        mean_s.append(mean)
        var_s.append(var)
        losses.append(loss.item())

        model.optimizer.zero_grad()
        loss.backward()
        model.optimizer.step()
    # if t % 100 == 99:
    print(t, losses)
    
    means = torch.stack(mean_s).mean(dim=0)
    varations = (torch.stack(var_s) + torch.stack(mean_s).pow(2)).mean(dim=0) - means.pow(2)



0 [0.28926336765289307, 0.2958078384399414, 0.29998689889907837, 0.29491499066352844, 0.30113136768341064]
1 [153436288.0, 701181824.0, 239045152.0, 506546592.0, 798447680.0]
2 [63.31367874145508, 8.236886024475098, 126.96015167236328, 58.33907699584961, 7.639318466186523]
3 [0.49102604389190674, 0.24390457570552826, 1.183990240097046, 0.7090784311294556, 0.30574512481689453]
4 [0.5693792700767517, 0.27500084042549133, 1.785478115081787, 0.6826379299163818, 0.32780107855796814]
5 [0.5540248155593872, 0.26137736439704895, 2.549373149871826, 0.6409949660301208, 0.3097911775112152]
6 [0.4617711305618286, 0.2597014605998993, 2.9763715267181396, 0.48002487421035767, 0.2689068019390106]
7 [0.2566162645816803, 0.2506767511367798, 2.8291988372802734, 0.32741427421569824, 0.23270846903324127]
8 [0.2877691686153412, 0.2336021512746811, 2.1497137546539307, 0.5568141341209412, 0.22389736771583557]
9 [0.2545499801635742, 0.2117825448513031, 1.2089921236038208, 0.4004654586315155, 0.2200218141078949

KeyboardInterrupt: ignored